In [1]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_file
from bokeh.models import Band, ColumnDataSource, HoverTool, DatetimeTickFormatter
from bokeh.models.annotations import Title

In [2]:
import random

def plot_lines(data_file, output_file, depth_type, start_date, end_date, num_outliers, use_top50=False):
    data = pd.read_csv(data_file)
    data = data.set_index(pd.date_range(start=start_date, end=end_date).date)
    data = data.transpose()
    data = data.set_index(pd.date_range(start=start_date, periods=24, freq='H').time)

    data_outputs = pd.read_csv(output_file).set_index(pd.date_range(start=start_date, end=end_date).date)

    data_outputs = data_outputs.transpose()

    data_final = pd.concat([data,data_outputs])

    data_final = data_final.sort_values(by=depth_type,axis=1)
    
    data_ordered = data_final
    
    color_top50 = ["#fbb4b9"] * (data_ordered.shape[1] - (num_outliers+1))
    color_median = ["Blue"]
    color_outliers = ["Red"] * num_outliers
    color_list = color_top50 + color_outliers + color_median
    '''
    color_list = []
    for i in range(data_ordered.shape[1]):
      color_list.append("#{:06x}".format(random.randint(0, 0xFFFFFF)))
    '''
    
    alpha_list = [0.5] * data_final.shape[1]
    alpha_list[0:num_outliers] = [1] * num_outliers
    alpha_list[len(alpha_list)-1] = 1

    numlines = len(data_ordered.columns)

    p = figure(width=1000, height=800, x_axis_type="datetime") 
    p.multi_line(xs=[data_ordered.index.values]*numlines,
                 ys=[data_ordered[name].values for name in data_ordered.iloc[0:24]],
                 line_color=color_list,
                 line_width=5)
    
    p.xaxis.major_label_text_font_size = "18pt"
    p.xaxis.formatter = DatetimeTickFormatter(days="%H:%M",hours="%H:%M")
    p.yaxis.major_label_text_font_size = "18pt"
    
    return p

In [4]:
output_file('taxis_v3_top50.html')
plot_taxis_v3 = plot_lines('../data/taxis_v3.csv',"../outputs/taxis_v3_out.txt",'tmd','1/1/2014','12/31/2018', 20, use_top50=False)
show(plot_taxis_v3)

In [53]:
output_file('taxis_v3_full.html')
plot_taxis_v3 = plot_lines('../data/taxis_v3.csv',"../outputs/taxis_v3_out.txt",'tmd','1/1/2014','12/31/2018', 20)
show(plot_taxis_v3)

(36, 1826)


In [54]:
output_file('taxis_v2_top50.html')
plot_taxis_v2 = plot_lines('../data/taxis_v2.csv',"../outputs/taxis_v2_out.txt",'tmd','1/1/2016','12/31/2018', 20, use_top50=True)
show(plot_taxis_v2)

(36, 568)


In [55]:
output_file('taxis_v2_full.html')
plot_taxis_v2 = plot_lines('../data/taxis_v2.csv',"../outputs/taxis_v2_out.txt",'tmd','1/1/2016','12/31/2018', 20)
show(plot_taxis_v2)

(36, 1096)


In [56]:
output_file('taxis_v1_top50.html')
plot_taxis_v1 = plot_lines('../data/taxis_v1.csv',"../outputs/taxis_v1_out.txt",'tmd','1/1/2018','12/31/2018', 20, use_top50=True)
show(plot_taxis_v1)

(36, 203)


In [30]:
output_file('taxis_v1_full.html')
plot_taxis_v1 = plot_lines('../data/taxis_v1.csv',"../outputs/taxis_v1_out.txt",'td','1/1/2018','12/31/2018', 20)
show(plot_taxis_v1)

In [31]:
output_file('taxis_v1_sliding.html')
plot_taxis_v1 = plot_lines('../data/taxis_v1.csv',"../outputs/taxis_v1_sliding_out.txt",'sd','1/1/2018','12/31/2018', 20)
show(plot_taxis_v1)

In [58]:
output_file('airquality_v1_top50.html')
plot_airquality_v1 = plot_lines('../data/airquality_v1.csv',"../outputs/airquality_v1_out.txt",'fd','03/10/2004','04/04/2005',20, use_top50=True)
show(plot_airquality_v1)

(36, 216)


In [59]:
output_file('airquality_v1_full.html')
plot_airquality_v1 = plot_lines('../data/airquality_v1.csv',"../outputs/airquality_v1_out.txt",'fd','03/10/2004','04/04/2005',20)
show(plot_airquality_v1)

(36, 391)


# Functional Boxplot

In [3]:
def prepare_data(data_file, output_file, start_date, end_date, index_days=None):
    data = pd.read_csv(data_file)
    if index_days:
        print(a)
    else:
        data = data.set_index(pd.date_range(start=start_date, end=end_date).date)
    data = data.transpose()
    data = data.set_index(pd.date_range(start=start_date, periods=24, freq='H').time)

    data_outputs = pd.read_csv(output_file).set_index(pd.date_range(start=start_date, end=end_date).date)
    data_outputs = data_outputs.transpose()

    data_final   = pd.concat([data,data_outputs])
    return data_final

def get_envelopes(data_top50):
    df_max = data_top50.max(axis=1)
    df_min = data_top50.min(axis=1)
    
    iqr = df_max - df_min
    mid = (df_max + df_min)//2
    out_top = mid + (0.75*iqr)
    out_bot = mid - (0.75*iqr)
    
    return pd.concat({'top':df_max,'bot':df_min,'out_top':out_top,'out_bot':out_bot},axis=1)

def get_outliers(raw_outliers, data_envelopes):
    data_outliers = pd.DataFrame()
    for col in raw_outliers.columns:
        if (raw_outliers[col][:24] > data_envelopes['out_top'][:24]).any() or (raw_outliers[col][:24] < data_envelopes['out_bot'][:24]).any():
            data_outliers = pd.concat([data_outliers,raw_outliers[col]],axis=1)
    return data_outliers

def functional_boxplot(data_file, output_file, start_date, end_date, depth_type):

    data_final   = prepare_data(data_file, output_file, start_date, end_date)
    data_final   = data_final.sort_values(by=depth_type,axis=1)
    
    data_median    = data_final.iloc[:,data_final.shape[1]-1]
    data_top50     = data_final.iloc[:,(data_final.shape[1]//2):data_final.shape[1]-1]
    data_envelopes = get_envelopes(data_top50)
    '''
    raw_outliers   = data_final.iloc[:,:(data_final.shape[1]//2)]
    data_outliers  = get_outliers(raw_outliers,data_envelopes)
    print(raw_outliers.shape)
    print(data_outliers.shape)
    '''
    data_ordered   = pd.concat([data_envelopes,data_median],axis=1)

    color_list = ["#f768a1","black","black","#f768a1"]+["blue"]
    alpha_list = [1.0]*4 + [1.0]
    
    numlines = len(data_ordered.columns)

    p = figure(width=1000, height=800, x_axis_type="datetime") 
    
    upper_band = np.array(data_envelopes['top'].iloc[0:24])
    lower_band = np.array(data_envelopes['bot'].iloc[0:24])
    x          = np.array(data_ordered.index[0:24])
    
    xs = np.concatenate([x, x[::-1]])
    ys = np.concatenate([lower_band, upper_band[::-1]])
    
    p.patch(x=xs, y=ys, fill_color="#f768a1", fill_alpha=0.8, line_alpha=0, legend="IQR")
    
    p.multi_line(xs=[data_ordered.index.values]*numlines,
                 ys=[data_ordered[name].values for name in data_ordered.iloc[0:24]],
                 line_color=color_list,
                 line_width=5,
                 alpha=alpha_list)
    
    p.xaxis.major_label_text_font_size = "18pt"
    p.xaxis.formatter = DatetimeTickFormatter(days="%H:%M",hours="%H:%M")
    p.yaxis.major_label_text_font_size = "18pt"

    return p

## Plot 1 year of taxis data

In [7]:
output_file('fbplot_taxisv1.html')
fbplot_taxisv1 = functional_boxplot('../data/taxis_v1.csv',"../outputs/taxis_v1_out.txt",'01/01/2018','12/31/2018','omd')
show(fbplot_taxisv1)

In [6]:
output_file('fbplot_taxisv1_s.html')
fbplot_taxisv1_s = functional_boxplot('../data/taxis_v1.csv',"../outputs/taxis_v1_sliding_out.txt",'01/01/2018','12/31/2018','sd')
show(fbplot_taxisv1_s)

## Plot 1 year of air quality data

In [152]:
output_file('fbplot_airquality_v1.html')
fbplot_taxisv1 = functional_boxplot('../data/airquality_v1.csv',"../outputs/airquality_v1_out.txt",'03/10/2004','04/04/2005','fd')
show(fbplot_taxisv1)

/home/pedro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



(36, 195)
(36, 100)


# Split datasets into weekdays and weekends

In [4]:
def split_datasets(data_file, output_file, start_date, end_date):

    data_aux = prepare_data(data_file, output_file, start_date, end_date)

    aux = []
    for col in data_aux.columns:
        aux.append(col.isoweekday())

    data_aux = data_aux.transpose()
    data_aux["weekday"] = aux
    data_weekends = data_aux.loc[data_aux["weekday"] > 5].drop("weekday",axis=1).transpose()
    data_weekdays = data_aux.loc[data_aux["weekday"] < 6].drop("weekday",axis=1).transpose()
    return data_weekdays, data_weekends

data_weekdays_taxi, data_weekends_taxi = split_datasets('../data/taxis_v1.csv',"../outputs/taxis_v1_out.txt",'01/01/2018','12/31/2018')
data_weekdays_airquality, data_weekends_airquality = split_datasets('../data/airquality_v1.csv',"../outputs/airquality_v1_out.txt",'03/10/2004','04/04/2005')

print(data_weekdays_taxi.shape, data_weekends_taxi.shape)
print(data_weekdays_airquality.shape, data_weekends_airquality.shape)

(36, 261) (36, 104)
(36, 279) (36, 112)


In [5]:
def functional_boxplot_from_df(data, depth_type, title):
    
    data_final   = data.sort_values(by=depth_type,axis=1)
    
    print(data_final.shape)
    
    data_median    = data_final.iloc[:,data_final.shape[1]-1]
    data_top50     = data_final.iloc[:,(data_final.shape[1]//2):data_final.shape[1]-1]
    print(data_top50.shape)
    data_envelopes = get_envelopes(data_top50)
    #print(data_envelopes)
    data_inner_envelopes = data_envelopes[['bot','top']]
    data_outer_envelopes = data_envelopes[['out_bot','out_top']]
    raw_outliers   = data_final.iloc[:,:(data_final.shape[1]//2)]
    #print(raw_outliers)
    data_outliers  = get_outliers(raw_outliers,data_envelopes)
    #print(data_outliers)
    data_top_outliers = data_outliers.iloc[:,:5]
    data_outliers.drop(data_outliers.iloc[:,:5], inplace=True, axis=1)

    print(data_outliers.shape)
    print(data_top_outliers.shape)
    
    data_ordered   = pd.concat([data_inner_envelopes,data_outliers,data_median],axis=1)
    #print(data_ordered.shape)
    
    color_inner_envelopes = ["#cccccc","#cccccc"]
    color_outliers = ["#fcbba1"]*(data_outliers.shape[1])
    color_median = ["black"]
    colors_general = color_inner_envelopes + color_outliers + color_median 
    
    numlines_general = len(data_ordered.columns)
    
    
    colors_outer_envelopes = ["#969696","#969696"]
    numlines_outer_envelopes = len(data_outer_envelopes.columns)

    
    colors_top_outliers = ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00']
    numlines_top_outliers = len(data_top_outliers.columns)    
    print(pd.to_datetime(data_top_outliers.columns))
    
    p = figure(width=1000, height=800, x_axis_type="datetime", title=title) 
    
    upper_band = np.array(data_envelopes['top'].iloc[0:24])
    lower_band = np.array(data_envelopes['bot'].iloc[0:24])
    x          = np.array(data_ordered.index[0:24])
    
    xs = np.concatenate([x, x[::-1]])
    ys = np.concatenate([lower_band, upper_band[::-1]])
    
    p.patch(x=xs, y=ys, fill_color="#cccccc", fill_alpha=0.8, line_alpha=0, legend="IQR")    
    
    src_outer_envelopes = ColumnDataSource(data={
        'xs_outer_envelopes': [data_outer_envelopes.index.values]*numlines_outer_envelopes,
        'ys_outer_envelopes': [data_outer_envelopes[name].values for name in data_outer_envelopes.iloc[0:24]],
        'colors_outer_envelopes':colors_outer_envelopes,
        'dates':data_outer_envelopes.columns.astype(str)
    })
    
    p.multi_line('xs_outer_envelopes',
                 'ys_outer_envelopes',
                 source=src_outer_envelopes,
                 line_color='colors_outer_envelopes',
                 line_width=5,
                 line_dash='dashed')
    
    src_general = ColumnDataSource(data={
        'xs_general': [data_ordered.index.values]*numlines_general,
        'ys_general': [data_ordered[name].values for name in data_ordered.iloc[0:24]],
        'colors_general': colors_general,
        'dates':data_ordered.columns.astype(str)
    })
    
    p.multi_line('xs_general',
                 'ys_general',
                 source=src_general,
                 line_color='colors_general',
                 line_width=5,
                 line_alpha=0.6,
                 hover_line_color='colors_general',
                 hover_line_alpha=1.0)
    
    src_top_outliers = ColumnDataSource(data={
        'xs_top_outliers': [data_top_outliers.index.values]*numlines_top_outliers,
        'ys_top_outliers': [data_top_outliers[name].values for name in data_top_outliers.iloc[0:24]],
        'colors_top_outliers': colors_top_outliers,
        'dates': data_top_outliers.columns.astype(str)
    })
    
    p.multi_line('xs_top_outliers',
                 'ys_top_outliers',
                 source=src_top_outliers,
                 line_color='colors_top_outliers',
                 line_width=5,
                 line_alpha=0.6,
                 hover_line_color='colors_top_outliers',
                 hover_line_alpha=1.0,
                 legend='dates',)
    
    p.legend.location = 'bottom_right'
    p.legend.background_fill_alpha = 0.5
    
    p.title.text_font_size = '20pt'
    
    p.xaxis.major_label_text_font_size = "18pt"
    p.xaxis.formatter = DatetimeTickFormatter(days="%H:%M",hours="%H:%M")
    p.yaxis.major_label_text_font_size = "18pt"
    
    p.add_tools(HoverTool(show_arrow=False, line_policy='next', tooltips=[
        ('Date', '@dates'),
    ]))
    
    return p

In [6]:
output_file('fbplot_ed_taxisv1.html')
ed_data = prepare_data('../data/taxis_v1.csv', "../outputs/taxis_v1_ed_out.txt",'01/01/2018','12/31/2018')
fbplot_taxisv1_ed = functional_boxplot_from_df(ed_data,'ed', 'Taxi trips using extremal depth')
show(fbplot_taxisv1_ed)

(39, 365)
(39, 182)
(39, 30)
(39, 5)
DatetimeIndex(['2018-01-04', '2018-12-25', '2018-03-11', '2018-02-02',
               '2018-01-01'],
              dtype='datetime64[ns]', freq=None)


In [7]:
output_file('sdepth/fbplot_depth.html')
data = prepare_data('../data/taxis_v1.csv', "../outputs/taxis_v1_out.txt",'01/01/2018','12/31/2018')
fbplot_taxisv1 = functional_boxplot_from_df(data, 'omd', 'Taxi trips using modified depth')
show(fbplot_taxisv1)

(36, 365)
(36, 182)
(36, 53)
(36, 5)
DatetimeIndex(['2018-01-01', '2018-12-25', '2018-12-26', '2018-08-19',
               '2018-12-30'],
              dtype='datetime64[ns]', freq=None)


In [11]:
output_file('sdepth/fbplot_sdepth_w7.html')
data_s7 = prepare_data('../data/taxis_v1.csv', "../outputs/taxis_v1_s7_out.txt",'01/01/2018','12/31/2018')
fbplot_taxisv1_s7 = functional_boxplot_from_df(data_s7, 'sd', 'Taxi trips using sliding depth with winsize 7')
show(fbplot_taxisv1_s7)

(38, 365)
(38, 182)


/home/pedro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



(38, 1)
(38, 5)
DatetimeIndex(['2018-07-04', '2018-12-25', '2018-11-22', '2018-03-21',
               '2018-01-04'],
              dtype='datetime64[ns]', freq=None)


In [12]:
output_file('sdepth/fbplot_sdepth_w15.html')
data_s15 = prepare_data('../data/taxis_v1.csv', "../outputs/taxis_v1_s15_out.txt",'01/01/2018','12/31/2018')
fbplot_taxisv1_s15 = functional_boxplot_from_df(data_s15, 'sd', 'Taxi trips using sliding depth with winsize 15')
show(fbplot_taxisv1_s15)

(38, 365)
(38, 182)


/home/pedro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



(38, 12)
(38, 5)
DatetimeIndex(['2018-12-16', '2018-12-25', '2018-01-28', '2018-01-01',
               '2018-02-25'],
              dtype='datetime64[ns]', freq=None)


In [13]:
output_file('sdepth/fbplot_sdepth_w31.html')
data_s31 = prepare_data('../data/taxis_v1.csv', "../outputs/taxis_v1_s31_out.txt",'01/01/2018','12/31/2018')
fbplot_taxisv1_s31 = functional_boxplot_from_df(data_s31, 'sd', 'Taxi trips using sliding depth with winsize 31')
show(fbplot_taxisv1_s31)

(38, 365)
(38, 182)


/home/pedro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



(38, 10)
(38, 5)
DatetimeIndex(['2018-01-01', '2018-12-25', '2018-07-04', '2018-01-28',
               '2018-11-22'],
              dtype='datetime64[ns]', freq=None)


In [14]:
output_file('sdepth/fbplot_sdepth_w61.html')
data_s61 = prepare_data('../data/taxis_v1.csv', "../outputs/taxis_v1_s61_out.txt",'01/01/2018','12/31/2018')
fbplot_taxisv1_s61 = functional_boxplot_from_df(data_s61, 'sd', 'Taxi trips using sliding depth with winsize 61')
show(fbplot_taxisv1_s61)

(38, 365)
(38, 182)


/home/pedro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



(38, 28)
(38, 5)
DatetimeIndex(['2018-01-01', '2018-12-25', '2018-05-28', '2018-05-27',
               '2018-09-02'],
              dtype='datetime64[ns]', freq=None)


# Plotting weekdays and weekends original depth

In [27]:
fbplot_taxisv1_weekday_od = functional_boxplot_from_df(data_weekdays_taxi,'od','Taxi trips on weekdays using original depth')
show(fbplot_taxisv1_weekday_od)
fbplot_taxisv1_weekday_fd = functional_boxplot_from_df(data_weekdays_taxi,'fd','Taxi trips on weekdays using rank matrix depth')
show(fbplot_taxisv1_weekday_fd)
fbplot_taxisv1_weekday_td = functional_boxplot_from_df(data_weekdays_taxi,'td','Taxi trips on weekdays using t-digest depth')
show(fbplot_taxisv1_weekday_td)


(36, 261)
(36, 130)
(36, 5)
(36, 5)
DatetimeIndex(['2018-01-01', '2018-04-03', '2018-04-19', '2018-05-31',
               '2018-01-04'],
              dtype='datetime64[ns]', freq=None)


/home/pedro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



(36, 261)
(36, 130)


/home/pedro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



(36, 18)
(36, 5)
DatetimeIndex(['2018-01-01', '2018-03-20', '2018-04-19', '2018-12-25',
               '2018-04-25'],
              dtype='datetime64[ns]', freq=None)
(36, 261)
(36, 130)


/home/pedro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



(36, 18)
(36, 5)
DatetimeIndex(['2018-01-01', '2018-03-20', '2018-04-19', '2018-12-25',
               '2018-04-25'],
              dtype='datetime64[ns]', freq=None)


In [6]:
fbplot_taxisv1_weekend_od = functional_boxplot_from_df(data_weekends_taxi,'od','Taxi trips on weekends using original depth')
show(fbplot_taxisv1_weekend_od)
fbplot_taxisv1_weekend_fd = functional_boxplot_from_df(data_weekends_taxi,'fd','Taxi trips on weekends using rank matrix depth')
show(fbplot_taxisv1_weekend_fd)
fbplot_taxisv1_weekend_td = functional_boxplot_from_df(data_weekends_taxi,'td','Taxi trips on weekends using t-digest depth')
show(fbplot_taxisv1_weekend_td)


/home/pedro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



(36, 20)
(36, 5)
DatetimeIndex(['2018-04-08', '2018-05-12', '2018-05-13', '2018-03-25',
               '2018-05-19'],
              dtype='datetime64[ns]', freq=None)


/home/pedro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



(36, 12)
(36, 5)
DatetimeIndex(['2018-02-24', '2018-03-11', '2018-11-04', '2018-05-13',
               '2018-02-10'],
              dtype='datetime64[ns]', freq=None)


/home/pedro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



(36, 12)
(36, 5)
DatetimeIndex(['2018-02-24', '2018-03-11', '2018-11-04', '2018-05-13',
               '2018-02-10'],
              dtype='datetime64[ns]', freq=None)


# Plotting weekdays and weekends modified depth

In [7]:
fbplot_taxisv1_weekday_omd = functional_boxplot_from_df(data_weekdays_taxi,'omd','Taxi trips on weekdays using original modified depth')
show(fbplot_taxisv1_weekday_omd)
fbplot_taxisv1_weekday_fmd = functional_boxplot_from_df(data_weekdays_taxi,'fmd','Taxi trips on weekdays using rank matrix modified depth')
show(fbplot_taxisv1_weekday_fmd)
fbplot_taxisv1_weekday_tmd = functional_boxplot_from_df(data_weekdays_taxi,'tmd','Taxi trips on weekdays using t-digest modified depth')
show(fbplot_taxisv1_weekday_tmd)


/home/pedro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



(36, 50)
(36, 5)
DatetimeIndex(['2018-01-01', '2018-12-25', '2018-12-26', '2018-01-04',
               '2018-07-04'],
              dtype='datetime64[ns]', freq=None)


/home/pedro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



(36, 50)
(36, 5)
DatetimeIndex(['2018-01-01', '2018-12-25', '2018-12-26', '2018-01-04',
               '2018-07-04'],
              dtype='datetime64[ns]', freq=None)


/home/pedro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



(36, 51)
(36, 5)
DatetimeIndex(['2018-01-01', '2018-12-25', '2018-12-26', '2018-01-04',
               '2018-07-04'],
              dtype='datetime64[ns]', freq=None)


In [21]:
from bokeh.io import export_svgs
fbplot_taxisv1_weekday_tmd = functional_boxplot_from_df(data_weekdays_taxi,'tmd','Taxi trips on weekdays using t-digest modified depth')
fbplot_taxisv1_weekday_tmd.output_backend = 'svg'
export_svgs(fbplot_taxisv1_weekday_tmd, filename="fbplot_taxisv1_weekday_tmd.svg")

/home/pedro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



(36, 51)
(36, 5)
DatetimeIndex(['2018-01-01', '2018-12-25', '2018-12-26', '2018-01-04',
               '2018-07-04'],
              dtype='datetime64[ns]', freq=None)


/home/pedro/anaconda3/lib/python3.6/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


['fbplot_taxisv1_weekday_tmd.svg']

In [8]:
fbplot_taxisv1_weekend_omd = functional_boxplot_from_df(data_weekends_taxi,'omd','Taxi trips on weekends using original modified depth')
show(fbplot_taxisv1_weekend_omd)
fbplot_taxisv1_weekend_fmd = functional_boxplot_from_df(data_weekends_taxi,'fmd','Taxi trips on weekends using rank matrix modified depth')
show(fbplot_taxisv1_weekend_fmd)
fbplot_taxisv1_weekend_tmd = functional_boxplot_from_df(data_weekends_taxi,'tmd','Taxi trips on weekends using t-digest modified depth')
show(fbplot_taxisv1_weekend_tmd)


/home/pedro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



(36, 11)
(36, 5)
DatetimeIndex(['2018-08-19', '2018-12-30', '2018-09-02', '2018-07-08',
               '2018-08-26'],
              dtype='datetime64[ns]', freq=None)


/home/pedro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



(36, 11)
(36, 5)
DatetimeIndex(['2018-08-19', '2018-12-30', '2018-09-02', '2018-07-08',
               '2018-08-26'],
              dtype='datetime64[ns]', freq=None)


/home/pedro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



(36, 11)
(36, 5)
DatetimeIndex(['2018-08-19', '2018-12-30', '2018-09-02', '2018-07-08',
               '2018-08-26'],
              dtype='datetime64[ns]', freq=None)


In [185]:
fbplot_aqv1_weekend = functional_boxplot_from_df(data_weekends_airquality,'od')
fbplot_aqv1_weekday = functional_boxplot_from_df(data_weekdays_airquality,'od')


/home/pedro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/pedro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



(36, 56)
(36, 13)
median: 2004-05-09
(36, 139)
(36, 64)
median: 2004-04-20


In [186]:
show(fbplot_aqv1_weekday)
show(fbplot_aqv1_weekend)